# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

This code handle multiple tools call by the llm model

In [3]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [4]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [5]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [6]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

#gr.ChatInterface(fn=chat, type="messages").launch()

## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [7]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [8]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [9]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [10]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

In [11]:
# Let's start by making a useful function

available_flight = {"pune": "Yes", "mumbai": "Yes", "tokyo": "No", "berlin": "No"}

def get_available_flight(destination_city):
    print(f"Tool get_available_flight called for {destination_city}")
    city = destination_city.lower()
    return available_flight.get(city, "Unknown")

In [12]:
get_available_flight("pune")

Tool get_available_flight called for pune


'Yes'

In [13]:
# There's a particular dictionary structure that's required to describe our function:

available_function = {
    "name": "get_available_flight",
    "description": "Get the availability of flight to the destination city. Call this whenever you need to know the flight availability, for example when a customer asks 'Do you have flight available to Pune'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [14]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},{"type": "function", "function": available_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [23]:
def chat(message, history):
    """
    Main chat function that handles the conversation flow, including tool calls.
    """
    # Append the user's message to the history
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    # First call to the model
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    response_message = response.choices[0].message

    # Check if the model decided to call tools
    if response_message.tool_calls:
        print("Model wants to call one or more tools.")
        # Append the assistant's message indicating a tool call is needed
        messages.append(response_message)

        # Execute the tool calls and get the responses
        tool_responses = handle_tool_call(response_message)

        # Append the tool responses to the conversation history
        messages.extend(tool_responses)

        # Make a second call to the model with the tool responses
        print("Sending tool responses back to the model...")
        second_response = openai.chat.completions.create(model=MODEL, messages=messages)

        # The final reply is in the second response
        final_response = second_response.choices[0].message.content
    else:
        # If no tool calls, the first response is the final one
        final_response = response_message.content

    return final_response

In [24]:
import gradio as gr

In [25]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_responses = []
    
    for tool_call in message.tool_calls:
        function_name=tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)


        print(f"Model requested call to function: '{function_name}' with arguments: {arguments}")

        
        if (function_name == 'get_ticket_price'):
            city = arguments.get('destination_city')
            price = get_ticket_price(city)
            content = json.dumps({"destination_city": city, "price": price})
            
        elif (function_name == 'get_available_flight'):
            city = arguments.get('destination_city')
            availability = get_available_flight(city)
            content = json.dumps({"destination_city": city, "available": availability})

        else:
            # Handle cases where the model calls a function that doesn't exist
            print(f"Error: Model called unknown function '{function_name}'")
            content = json.dumps({"error": "Unknown function called."})

        # Append the formatted response for this specific tool call
        tool_responses.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name": function_name,
            "content": content,
        })

    return tool_responses

In [26]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


Model wants to call one or more tools.
Model requested call to function: 'get_available_flight' with arguments: {'destination_city': 'Pune'}
Tool get_available_flight called for Pune
Sending tool responses back to the model...
Model wants to call one or more tools.
Model requested call to function: 'get_ticket_price' with arguments: {'destination_city': 'Tokyo'}
Tool get_ticket_price called for Tokyo
Sending tool responses back to the model...
Model wants to call one or more tools.
Model requested call to function: 'get_available_flight' with arguments: {'destination_city': 'Pune'}
Tool get_available_flight called for Pune
Model requested call to function: 'get_ticket_price' with arguments: {'destination_city': 'Tokyo'}
Tool get_ticket_price called for Tokyo
Sending tool responses back to the model...
